In [ ]:
import os
import openai
import configparser
import speech_recognition as sr
import pyaudio
import wave
import keyboard

In [ ]:
#API key setting

config = configparser.ConfigParser()
config.read('config.ini')
openai.api_key = config.get('api', 'api_key')

In [ ]:
# GPT-3 API request

def GPT_API(input):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt= input,
        max_tokens=200
    )

    return response.choices[0].text.strip()

# Whisper API request
def Whisper_API(input_path):
    audio_file = open(input_path, 'rb')
    transcript = openai.Audio.transcribe('whisper-1', audio_file)
    
    return transcript['text']

#voice recode

def voice_recode():
    FORMAT = pyaudio.paInt16  
    CHANNELS = 1  
    RATE = 44100  
    CHUNK = 1024  
    OUTPUT_DIRECTORY = "record"
    OUTPUT_FILENAME = os.path.join(OUTPUT_DIRECTORY, "recorded_audio_ver1.wav")

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

    print("Recording Start --> (if press 'q', quit recording )")

    frames = []

    while not keyboard.is_pressed('q'):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording Stop!")

    # 스트림과 PyAudio 객체 닫기
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # 녹음된 데이터를 WAV 파일로 저장
    with wave.open(OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"녹음된 파일 {OUTPUT_FILENAME} 저장 완료.")


In [ ]:
voice_recode()
text = Whisper_API('./record/recorded_audio.wav')
result = GPT_API(text)
result